<a href="https://colab.research.google.com/github/SarahGoddaer/Machine_Learning_course_UGent_D012554_kaggle/blob/master/9%3A%20Voting_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn import metrics

Here, I want to know if combining the different results of the different models I used can make an improvement towards predicting test set labels.

#Voting classifier

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")
testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

features = trainset.copy()
features.pop('label')
feature_names = list(features.columns)

test_features = testset.copy()
test_features.pop('index')
test_feature_names = list(test_features.columns)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# Training classifiers
clf1 = make_pipeline(StandardScaler(), SVC(C=10,gamma=1, probability=True))
clf2 = RandomForestClassifier(criterion='entropy', n_estimators=250, 
                              max_depth=30, random_state=0)
clf3 = XGBClassifier(n_estimators= 900, gamma=0, max_depth=7, 
                     min_child_weight=0, learning_rate=0.1, 
                     subsample=0.85, colsample_bytree=0.9)
clf4 = KNeighborsClassifier(n_neighbors=4, weights='distance')
clf5 = make_pipeline(StandardScaler(),LogisticRegression(C=100))

eclf = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4), 
                                    ('reg', clf5)], voting='soft')


In [17]:
for clf, name in zip([clf1, clf2, clf3, clf4, clf5, eclf], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Logistic regression', 'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=10)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96253 (+/- 0.01182) [SVC]
AUC score: 0.93205 (+/- 0.01971) [Random Forest]
AUC score: 0.94602 (+/- 0.02002) [XGBoost]
AUC score: 0.95938 (+/- 0.01119) [KNeighbors]
AUC score: 0.65963 (+/- 0.03643) [Logistic regression]
AUC score: 0.96706 (+/- 0.01412) [Ensemble]


Ok, this seems to work! Now I will see if this approach maybe better works without the logistic regression model...

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# Training classifiers
clf1 = make_pipeline(StandardScaler(), SVC(C=10,gamma=1, probability=True))
clf2 = RandomForestClassifier(criterion='entropy', n_estimators=250, 
                              max_depth=30, random_state=0)
clf3 = XGBClassifier(n_estimators= 900, gamma=0, max_depth=7, 
                     min_child_weight=0, learning_rate=0.1, 
                     subsample=0.85, colsample_bytree=0.9)
clf4 = KNeighborsClassifier(n_neighbors=4, weights='distance')

eclf = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

In [19]:
for clf, name in zip([clf1, clf2, clf3, clf4, eclf], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=10)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96253 (+/- 0.01182) [SVC]
AUC score: 0.93205 (+/- 0.01971) [Random Forest]
AUC score: 0.94602 (+/- 0.02002) [XGBoost]
AUC score: 0.95938 (+/- 0.01119) [KNeighbors]
AUC score: 0.96951 (+/- 0.01216) [Ensemble]
